In [1]:
import os
from experiment_setup import setups

In [2]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/coveo_ecommerce"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [3]:
params = setups["coveo"]["params_bprmax"]

In [4]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [10]:
def create_recpack_script(model_name, train_path, test_path, loss, optim, layers, embedding, hidden_size, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, bpreg, n_epochs, n_sample, m, use_correct_weight_init):
    s_train_test_full = (
        f"python ../recpack/main.py --train_path {train_path} "
        f"--test_path {test_path} --m {m} --n_epochs {n_epochs} --loss {loss} --optim {optim} "
        f"--layers {layers} --embedding {embedding} --hidden_size {hidden_size} "
        f"--batch_size {batch_size} --dropout_p_embed {dropout_p_embed} "
        f"--dropout_p_hidden {dropout_p_hidden} --learning_rate {learning_rate}"
        f"{f' --n_sample {n_sample}' if loss=='bpr-max' else ''}"
        f"{' --use_correct_weight_init' if use_correct_weight_init else ''}"
        f"{f' --bpreg {bpreg}' if loss=='bpr-max' else ''}"
    )
    return s_train_test_full

In [6]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [11]:
train_test_script_oob = create_recpack_script(model_name='recpack_oob_bprmax', train_path=train_path, test_path=test_path,loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_embed, learning_rate=learning_rate, bpreg=1.0, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=False)

In [12]:
print(train_test_script_oob)

python ../recpack/main.py --train_path ../datasets/coveo_ecommerce\coveo_processed_view_train_full.tsv --test_path ../datasets/coveo_ecommerce\coveo_processed_view_test.tsv --m 1 5 10 20 --n_epochs 5 --loss bpr-max --optim adagrad --layers 1 --embedding 512 --hidden_size 512 --batch_size 144 --dropout_p_embed 0.35 --dropout_p_hidden 0.35 --learning_rate 0.05 --n_sample 2048 --bpreg 1.0


In [14]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_test_script_oob, "recpack_oob_bprmax", "coveo")

[codecarbon WARNING @ 19:07:56] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 19:07:56] [setup] RAM Tracking...
[codecarbon INFO @ 19:07:56] [setup] GPU Tracking...
[codecarbon INFO @ 19:07:56] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:07:57] [setup] CPU Tracking...
[codecarbon WARNING @ 19:07:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:07:59] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 19:07:59] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon INFO @ 19:07:59] >>> Tracker's metadata:
[codecarbon INFO @ 19:07:59]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 19:07:59]   Python version: 3.12.3
[codecarbon INFO @ 19:07:59]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 19:07:59]   Available RAM : 31.746 GB
[codecarbon INFO @ 19:07:59]   CPU count: 32
[codecarbon 

CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4090 Laptop GPU.
The training of models in this work is estimated to use 0.001 kWh of electricity contributing to 0.000 kg of CO2eq. This is equivalent to 0.004 km travelled by car. Measured by carbontracker (https://github.com/lfwa/carbontracker).
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 19:08:18] Energy consumed for RAM : 0.000051 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:08:18] Energy consumed for all GPUs : 0.000157 kWh. Total GPU Power : 36.63281697044478 W
[codecarbon INFO @ 19:08:18] Energy consumed for all CPUs : 0.000182 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:08:18] 0.000390 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:08:33] Energy consumed for RAM : 0.000101 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:08:33] Energy consumed for all GPUs : 0.000336 kWh. Total GPU Power : 43.08023929514272 W
[codecarbon INFO @ 19:08:33] Energy consumed for all CPUs : 0.000359 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:08:33] 0.000796 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:08:48] Energy consumed for RAM : 0.000150 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:08:48] Energy consumed for all GPUs : 0.000506 kWh. Total GPU Power : 40.83738459448

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 19:22:03] Energy consumed for RAM : 0.002779 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:22:03] Energy consumed for all GPUs : 0.010789 kWh. Total GPU Power : 50.387423774654636 W
[codecarbon INFO @ 19:22:03] Energy consumed for all CPUs : 0.009925 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:22:03] 0.023492 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:22:18] Energy consumed for RAM : 0.002828 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:22:18] Energy consumed for all GPUs : 0.010986 kWh. Total GPU Power : 47.54109344384474 W
[codecarbon INFO @ 19:22:18] Energy consumed for all CPUs : 0.010102 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:22:18] 0.023916 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:22:33] Energy consumed for RAM : 0.002878 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 19:22:33] Energy consumed for all GPUs : 0.011192 kWh. Total GPU Power : 49.2720921005

...

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


[codecarbon INFO @ 20:08:04] Energy consumed for RAM : 0.011904 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:08:04] Energy consumed for all GPUs : 0.044270 kWh. Total GPU Power : 46.20882682522554 W
[codecarbon INFO @ 20:08:04] Energy consumed for all CPUs : 0.042520 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:08:04] 0.098693 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:08:19] Energy consumed for RAM : 0.011953 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:08:19] Energy consumed for all GPUs : 0.044467 kWh. Total GPU Power : 47.31888864249815 W
[codecarbon INFO @ 20:08:19] Energy consumed for all CPUs : 0.042697 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:08:19] 0.099117 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:08:34] Energy consumed for RAM : 0.012003 kWh. RAM Power : 11.904736518859863 W
[codecarbon INFO @ 20:08:34] Energy consumed for all GPUs : 0.044655 kWh. Total GPU Power : 45.03846066183

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: 
Actual consumption for 1 epoch(s):
	Time:	1:01:33
	Energy:	0.072257573176 kWh
	CO2eq:	27.057804243487 g
	This is equivalent to:
	0.251700504591 km travelled by car
CarbonTracker: 
Actual consumption for 1 epoch(s):
	

0.03366463088163897

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


## Train & test the minor fix model

In [ ]:
train_test_script_minorfix = create_recpack_script(model_name='recpack_minorfix_bprmax', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_embed, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [ ]:
os.system(train_test_script_minorfix)

## Train & test the major fix model

In [ ]:
train_test_script_majorfix = create_recpack_script(model_name='recpack_majorfix_bprmax', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [ ]:
os.system(train_test_script_majorfix)